In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance as gd

from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta

pd.options.mode.chained_assignment = None

# from pandarallel import pandarallel

In [ ]:
vessel_information = pd.read_csv("./data/Vessel_information.csv")
list_vessel_id = vessel_information.loc[(vessel_information['type'] == 'CSV'), 'vessel_id'].to_list()

In [ ]:
df_vessel_daily_data = pd.read_pickle("./data/df_vessel_daily_2019_complete.pkl")
df_vessel_daily_data.loc[df_vessel_daily_data['time_offset'] == 'Z', 'time_offset'] = '00:00'
df_vessel_daily_data = df_vessel_daily_data.loc[df_vessel_daily_data.time_offset.isna() == False]

In [ ]:
df_positions_jan2019 = pd.read_csv("./data/positions_jan2019.csv")
df_positions_feb2019 = pd.read_csv("./data/positions_feb2019.csv")
df_positions_mar2019 = pd.read_csv("./data/positions_mar2019.csv")
df_positions_apr2019 = pd.read_csv("./data/positions_apr2019.csv")
df_positions_may2019 = pd.read_csv("./data/positions_may2019.csv")
df_positions_jun2019 = pd.read_csv("./data/positions_june2019.csv")
df_positions_jul2019 = pd.read_csv("./data/positions_july2019.csv")
df_positions_aug2019 = pd.read_csv("./data/positions_aug2019.csv")
df_positions_sep2019 = pd.read_csv("./data/positions_sept2019.csv")
df_positions_oct2019 = pd.read_csv("./data/positions_oct2019.csv")
df_positions_nov2019 = pd.read_csv("./data/positions_nov2019.csv")
df_positions_dec2019 = pd.read_csv("./data/positions_dec2019.csv")


list_df_position = [df_positions_jan2019, df_positions_feb2019, 
                    df_positions_mar2019, df_positions_apr2019, 
                    df_positions_may2019, df_positions_jun2019, 
                    df_positions_jul2019, df_positions_aug2019, 
                    df_positions_sep2019, df_positions_oct2019, 
                    df_positions_nov2019, df_positions_dec2019,
                   ]

df_positions_2019 = pd.concat(list_df_position, axis=0, ignore_index=True)


df_positions_2019 = df_positions_2019[(df_positions_2019['vessel_id'].isin(list_vessel_id))]
df_positions_2019["new_position_received_time"] = pd.to_datetime(df_positions_2019['position_received_time'])
df_positions_2019['new_position_received_time'] = df_positions_2019['new_position_received_time'].dt.tz_localize(None)

In [ ]:
df_positions_2019.drop_duplicates(subset=['vessel_id', 'course', 'destination', 'draught', 'heading',
       'latitude', 'longitude', 'nav_status', 'speed', 'eta_time',
       'position_received_time', 'location', 'api_source'], inplace=True)

In [ ]:
df_positions_2019.sort_values(by=['vessel_id', 'new_position_received_time'], inplace=True)

In [ ]:
for vessel_id in list_vessel_id:
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_lon'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'longitude'].shift(1)
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_lat'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'latitude'].shift(1)
    
    df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'prev_receive_time'] = df_positions_2019.loc[df_positions_2019['vessel_id']==vessel_id, 'new_position_received_time'].shift(1)

In [ ]:
df_positions_2019['prev_coord'] = df_positions_2019.apply(lambda x: (x.prev_lat, x.prev_lon), axis=1)
df_positions_2019['curr_coord'] = df_positions_2019.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [ ]:
# Here is the calculation of mile since the last record (current_coordinate - previous_coordinate)
# This process takes sometime to complete
df_positions_2019['mile_since'] = df_positions_2019.loc[df_positions_2019['prev_lon'].notnull()].apply(
    lambda x: gd.distance(x.prev_coord, x.curr_coord).nm, axis=1) 

df_positions_2019[['prev_coord', 'curr_coord', 'mile_since']].head(2)

In [ ]:
# Here is the calculation of hour since the last record (current_time - previous_time)
# Then convert to hourly format
df_positions_2019['hour_since'] = (df_positions_2019.new_position_received_time - df_positions_2019.prev_receive_time)/np.timedelta64(1, 'h')
df_positions_2019[['hour_since']].head(2)


In [ ]:
# Here is the calculation of speed (mile_since - hour_since) --> nautical_mile/hour
df_positions_2019.at[(df_positions_2019['hour_since'] == 0) & (df_positions_2019['mile_since'] > 0), 'hour_since']=0.001

df_positions_2019['speed_nm'] = (df_positions_2019.mile_since / df_positions_2019.hour_since)


In [ ]:
df_positions_2019.loc[(df_positions_2019['hour_since'] == 0) & (df_positions_2019['mile_since'] == 0), 'speed_nm'] = 0

In [ ]:
df_positions_2019.columns

In [ ]:
def apply_label_combine_transit(label, new_label):
    for idx, row in df_vessel_daily_data[(df_vessel_daily_data['activity'] == label)
                                            &
                                           (df_vessel_daily_data['vessel_id'].isin(list_vessel_id))].iterrows():
        consumption_at_period = row.fuel/row.report_hours
        df_positions_2019.loc[((df_positions_2019['new_position_received_time']+timedelta(hours=(int(row.time_offset.split(":")[0])), 
                                                              minutes=(int(row.time_offset.split(":")[1]))))
                              >= row.new_report_time-timedelta(hours=row.report_hours )) 
                             & 
                             ((df_positions_2019['new_position_received_time']+timedelta(hours=(int(row.time_offset.split(":")[0])), 
                                                              minutes=(int(row.time_offset.split(":")[1]))))
                              < row.new_report_time)
                             &
                             (df_positions_2019['vessel_id']== row.vessel_id)
                                 , ['activity_label', 'activity_label2', 'time_period', 'ref_date','transit_type', 'daily_vessel_id', 'fuel_consumption_average']] = [new_label, row.activity_mode, row.time_period, row.new_report_time, label, row.id, consumption_at_period]        

In [ ]:
def convert_activity_mode(row):
    data = {'activity_mode':['002 - Anchor Handling - Medium Main-eng.',
                             '011 - Towing - Manual',
                             '010 - Towing - DP Auto Pos',
                             'AH - Towing',
                             '003 - Anchor Handling - Heavy Tention'],
            'activity':['AH', 'Towing', 'Towing', 'Towing', 'AH']} 
    df_lookup = pd.DataFrame(data)
    df_temp = df_lookup.loc[df_lookup['activity_mode'] == row.activity_label2]
    if df_temp.values.size > 0:
        return df_temp.activity.values[0]
    else:
        return None

In [ ]:
# def calculate_fuel_consumption(related_dataframe):
# for idx, row in df_vessel_daily_data.loc[(df_vessel_daily_data['vessel_id'].isin(list_vessel_id))
#                                         ].iterrows():
# #     print(row.time_offset)
#     consumption_at_period = row.fuel/row.report_hours
#     df_positions_2019.loc[
#         ((df_positions_2019['new_position_received_time'] 
#           + timedelta(hours=(int(row.time_offset.split(":")[0])),
#                       minutes=(int(row.time_offset.split(":")[1])))) >= row.new_report_time-timedelta(hours=row.report_hours)) 
#         & 
#         ((df_positions_2019['new_position_received_time'] 
#           + timedelta(hours=(int(row.time_offset.split(":")[0])),
#                       minutes=(int(row.time_offset.split(":")[1])))) < row.new_report_time)
#         , 'fuel_consumption_average'] = consumption_at_period


In [ ]:
df_positions_2019.to_pickle("data/[CSV]df_positions_2019_ver1.pkl")

#### resize the df_positions (take out some not used columns, resize columns type) 

In [ ]:
df_positions_2019.columns

In [ ]:
df_positions_2019[['id', 'vessel_id']].apply(pd.to_numeric, errors='ignore', downcast='integer').info()
df_positions_2019[['course', 'draught', 'longitude', 'latitude',
                     'speed', 'prev_lon', 'prev_lat', 
                     'mile_since', 'hour_since', 'speed_nm']].apply(pd.to_numeric, errors='ignore', downcast='float').info()

In [ ]:
# pd.to_numeric(df_positions_90_2019[['id', 'vessel_id']], )
df_positions_2019['vessel_id'] = df_positions_2019['vessel_id'].astype('int16')
df_positions_2019['id'] = df_positions_2019['id'].astype('int32')

df_positions_2019[['course', 'draught', 'longitude', 'latitude',
                     'speed', 'prev_lon', 'prev_lat', 
                     'mile_since', 'hour_since', 'speed_nm']] = df_positions_2019[['course', 'draught', 'longitude', 'latitude',
                                                                       'speed', 'prev_lon', 'prev_lat', 
                                                                       'mile_since', 'hour_since', 'speed_nm']].astype('float32')

In [ ]:
df_positions_2019 = df_positions_2019[['id', 'vessel_id','speed', 
       'new_position_received_time', 
       'prev_receive_time', 'prev_coord', 'curr_coord', 'mile_since',
       'hour_since', 'speed_nm']]

In [ ]:
df_positions_2019.to_pickle("data/[resized]df_CSV_positions_2019.pkl")

In [ ]:
df_positions_2019 = pd.read_pickle("./data/[resized]df_CSV_positions_2019.pkl")

#### End of resizing block

#### Do the model still need this?

In [ ]:
df_positions_2019.loc[(df_positions_2019.fuel_consumption_average.isna()==False) 
                         &
                         (df_positions_2019.speed_nm.isna()==False), 'speed_consumption_average'] = df_positions_2019.loc[(df_positions_2019.fuel_consumption_average>0)
                                                                                                                                &
                                                                                                                                (df_positions_2019.speed_nm > 0)].apply(lambda x: x.speed_nm/x.fuel_consumption_average, axis=1)

In [ ]:
# For CSV type
apply_label_combine_transit('DP', 'DP')
apply_label_combine_transit('Standby','Standby')
apply_label_combine_transit('Transit Eco', 'TransitCombine')
apply_label_combine_transit('Transit', 'TransitCombine')
apply_label_combine_transit('Transit Max', 'TransitCombine')
apply_label_combine_transit('Port', 'Port')
apply_label_combine_transit('AH/Towing', 'AH/Towing')

In [ ]:
df_positions_2019.loc[(df_positions_2019['activity_label'] == 'AH/Towing'), 'activity_label'] = df_positions_2019.loc[(df_positions_2019['activity_label'] == 'AH/Towing')].apply(lambda x: convert_activity_mode(x), axis=1)